In [1]:
import pandas as pd

In [2]:
census=pd.read_csv('census_data.csv')

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [6]:
def label_fix(label):
    if label== ' <=50K':
        return 0
    else:
        return 1
    

In [7]:
census['income_bracket']=census['income_bracket'].apply(label_fix)

In [8]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_data=census.drop('income_bracket',axis=1)

In [11]:
y_labels=census['income_bracket']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

In [13]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [14]:
import tensorflow as tf

In [15]:
gender=tf.feature_column.categorical_column_with_vocabulary_list("gender",['Female','Male'])

In [17]:
occupation=tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=100)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [18]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [19]:
feat_cols=[gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [20]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [21]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\Satish\\AppData\\Local\\Temp\\tmpssai2jtp', '_tf_random_seed': 1, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5}


In [22]:
model.train(input_fn=input_func,steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Satish\AppData\Local\Temp\tmpssai2jtp\model.ckpt.
INFO:tensorflow:step = 1, loss = 69.3147
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\Satish\AppData\Local\Temp\tmpssai2jtp\model.ckpt.
INFO:tensorflow:Loss for final step: 4596.58.


In [23]:
pred_fn=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [24]:
pred_gen=model.predict(input_fn=pred_fn)

In [25]:
predictions=list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\Satish\AppData\Local\Temp\tmpssai2jtp\model.ckpt-100


In [28]:
final_pred=[pred['class_ids'][0] for pred in predictions]

In [29]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(y_test,final_pred))

             precision    recall  f1-score   support

          0       0.88      0.87      0.88      7436
          1       0.61      0.64      0.62      2333

avg / total       0.82      0.81      0.82      9769

